# Estudo Biblioteca Keras com Validação Cruzada utilizando o StratifiedKFold
- O objetivo deste notebook é o estudo da Biblioteca Keras.
- Keras é uma biblioteca para rede neural de alto-nível escrita em Python. Ela roda como frontend em TensorFlow ou Theano. 
- Uma das grandes vantagens de utilizar essa biblioteca é que podemos substituir uma rede neural por outra de maneira muito fácil, pois ela foi criada com intuito de tornar essas experimentações mais fáceis, rápidas e eficientes. Com o Keras você pode abstrair tudo que acontece em background e focar no seu problema.
- Mais detalhes na página do projeto disponível neste <a href="https://keras.io/">link</a>.

- Nesse teste além do Keras vamos utilizar o StratifiedKFold do Sklearn para realizar a validação cruzada do modelo.
- O StratifiedKFold é uma variação do KFold que retorna folds estratificados, ou seja, em cada fold é preservado a porcentagem de amostras para cada classe.
- Mais informações sobre o StratifiedKFold estão disponíveis neste <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold.split">link</a>.


<h3> Onde usar o Keras? </h3>

- Prototipagem rápida e fácil (total modularidade, minimalismo e extensibilidade).
- Suporte a redes convolucionais e recorrentes, incluindo combinação de ambas.
- Suporte a esquemas de conectividade arbitrária (incluindo treino de N para N).
- Rodar na CPU ou GPU.

<h3> DataSet </h3>
- Para este estudo vamos utilizar o data set 'census.csv'.</br>
- Este dataset contém 15 atributos e 32.561 instâncias (linhas). </br>


In [1]:
# Importando as bibliotecas iniciais
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Importa o dataset
base = pd.read_csv('./census.csv')

In [3]:
# Verificando o shape do dataset
base.shape

(32561, 15)

In [4]:
# Visualiza as primeira linhas
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
# Verificando os atributos núméricos.
base.describe()

,age,final-weight,education-num,capital-gain,capital-loos,hour-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


# Pré-processamento - Tratamento dos dados

In [6]:
# Seleciona os atributos previsores
previsores = base.iloc[:, 0:14].values
previsores

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [7]:
# Seleciona a classe alvo (target)
classe = base.iloc[:,14]
classe.head()

0     <=50K
1     <=50K
2     <=50K
3     <=50K
4     <=50K
Name: income, dtype: object

In [8]:
# Realiza a codificação dos atributos categóricos. Transforma as categorias em códigos.
from sklearn.preprocessing import LabelEncoder

label_encoder_previsores = LabelEncoder()
previsores[:,1] = label_encoder_previsores.fit_transform(previsores[:,1])
previsores[:,3] = label_encoder_previsores.fit_transform(previsores[:,3])
previsores[:,5] = label_encoder_previsores.fit_transform(previsores[:,5])
previsores[:,6] = label_encoder_previsores.fit_transform(previsores[:,6])
previsores[:,7] = label_encoder_previsores.fit_transform(previsores[:,7])
previsores[:,8] = label_encoder_previsores.fit_transform(previsores[:,8])
previsores[:,9] = label_encoder_previsores.fit_transform(previsores[:,9])
previsores[:,13] = label_encoder_previsores.fit_transform(previsores[:,13])
previsores

array([[39, 7, 77516, ..., 0, 40, 39],
       [50, 6, 83311, ..., 0, 13, 39],
       [38, 4, 215646, ..., 0, 40, 39],
       ...,
       [58, 4, 151910, ..., 0, 40, 39],
       [22, 4, 201490, ..., 0, 20, 39],
       [52, 5, 287927, ..., 0, 40, 39]], dtype=object)

In [9]:
labelencoder_classe = LabelEncoder()
classe = labelencoder_classe.fit_transform(classe)

In [10]:
# Realiza a binarização dos valores inteiros. Técnica chamada: Dummy variables.
# Isto é importante para que o algoritmo não considere um valor maior mais importante que um valor menor.
# Não tivemos ganhos com ele, por isso vamos deixar dasabilitado.

#from sklearn.preprocessing import OneHotEncoder

#oneHotEnconder = OneHotEncoder(categories='auto')
#previsores = oneHotEnconder.fit_transform(previsores).toarray()
#previsores

In [11]:
# StandardScaler para colocar todos os valores na mesma escala (escalonamento).
# Teste do escalonamente somente dos atributos inteiros, também tivemos o mesmo resultado da padronização total. Tivemos praticamente o mesma Acurácia com o escalonamento total. 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

previsores[:,0] =  scaler.fit_transform(previsores[:,0].reshape(-1,1)).ravel()
previsores[:,2] =  scaler.fit_transform(previsores[:,2].reshape(-1,1)).ravel()
previsores[:,4] =  scaler.fit_transform(previsores[:,4].reshape(-1,1)).ravel()
previsores[:,10] =  scaler.fit_transform(previsores[:,10].reshape(-1,1)).ravel()
previsores[:,11] =  scaler.fit_transform(previsores[:,11].reshape(-1,1)).ravel()
previsores[:,12] =  scaler.fit_transform(previsores[:,12].reshape(-1,1)).ravel()

In [12]:
# Separa os dados pata treinamento e teste
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe , test_size = 0.25, random_state = 0)


# Definindo o Modelo de Previsão

In [13]:
# Cria o modelo de previsão 
import keras
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')

def cria_modelo():

    # Cria camada de entrada da rede
    modelo = Sequential()

    # Cria a 1 camada oculta da rede
    modelo.add(Dense(units = 8, activation = 'relu', input_dim = 14))

    # Cria a 2 camada oculta da rede
    modelo.add(Dense(units = 8, activation = 'relu'))

    # Cria a camada de saida da rede
    modelo.add(Dense(units = 1, activation = 'sigmoid'))

    # Aplica as configurações no modelo para o treinamento.
    modelo.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return modelo

Using TensorFlow backend.


In [ ]:
#Define o tempo de inicio do Treinamento
inicio = datetime.now()

# Realizando a validação cruzada com StratifiedKFold

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

# Define a quantidade de folds que será utilizado (folds são "pedaços" da base de dados, neste caso a base será dividida em 10 "partes")
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 3)

acuracia_testes = []
matrizes_confusao = []
placeholder = np.zeros(shape=(previsores.shape[0], 1))
qtd_modelo = 0


for indice_treinamento, indice_teste in kfold.split(previsores, placeholder):
    
    print('\n ------------ Treinando modelo: %d ------------' % qtd_modelo)
    
    # Cria um instância do modelo para treinamento.
    classificador = cria_modelo()

    # Realiza o treinamento do modelo.
    classificador.fit(previsores[indice_treinamento], classe[indice_treinamento], batch_size = 10, epochs = 400)
    
    # Realiza o teste do modelo    
    previsoes = classificador.predict(previsores[indice_teste])
    
    previsoes = (previsoes > 0.5)
    
    # Adiciona a matrix de confusão do modelo na lista
    matrizes_confusao.append(confusion_matrix(classe[indice_teste], previsoes))
    
    # Adiciona a acurácia do modelo na lista
    acuracia_testes.append(accuracy_score(classe[indice_teste], previsoes))
    
    qtd_modelo +=1


W0808 09:32:24.243610 139669144549184 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 09:32:24.253973 139669144549184 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 09:32:24.255521 139669144549184 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 09:32:24.281527 139669144549184 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W080


 ------------ Treinando modelo: 0 ------------
Epoch 1/400
29304/29304 [==============================] - 2s 79us/step - loss: 0.4886 - acc: 0.7913: 0s - loss: 0.4986 - acc: 0
Epoch 2/400
29304/29304 [==============================] - 2s 74us/step - loss: 0.4126 - acc: 0.8102
Epoch 3/400
29304/29304 [==============================] - 2s 70us/step - loss: 0.4038 - acc: 0.8164
Epoch 4/400
29304/29304 [==============================] - 2s 74us/step - loss: 0.3984 - acc: 0.8224
Epoch 5/400
29304/29304 [==============================] - 2s 76us/step - loss: 0.3950 - acc: 0.8239
Epoch 6/400
29304/29304 [==============================] - 2s 68us/step - loss: 0.3920 - acc: 0.8252
Epoch 7/400
29304/29304 [==============================] - 2s 76us/step - loss: 0.3846 - acc: 0.8247
Epoch 8/400
29304/29304 [==============================] - 2s 68us/step - loss: 0.3608 - acc: 0.8347
Epoch 9/400
29304/29304 [==============================] - 2s 69us/step - loss: 0.3439 - acc: 0.8399
Epoch 10/400
29

In [ ]:
#Define o tempo final do Treinamento
fim = datetime.now()   
tempo_modelo = (fim - inicio).total_seconds()
print('\nTempo de Treinamento: %s segundos' % tempo_modelo )

# Verificando a performance do Modelo

In [15]:
# Analisando a performance do Modelo

acuracia_final = np.asarray(acuracia_testes)
print('\nMédia acurácia do Modelo: %.2f%% ' % (acuracia_final.mean()*100))

# Desvio padrão alto pode significar overfitting do modelo.
print('\nDesvio padrão média acurácia do Modelo: %.4f ' % acuracia_final.std())



Média acurácia do Modelo: 84.86% 

Desvio padrão média acurácia do Modelo: 0.0066 


In [16]:
# Visualizando a Matriz de confusão (confusion matrix)

# Calcula a média das matrizes de confusão de cada modelo gerado.
matriz_confusao_final = np.mean(matrizes_confusao, axis = 0)

df_cm = pd.DataFrame(map(np.round, matriz_confusao_final))

print('\n')
print('Obs.: Nas linhas temos as Classes reais e nas colunas as Classes previstas.\n')
print('Matriz de Confusão. *os valores foram arredondados.')

df_cm



Obs.: Nas linhas temos as Classes reais e nas colunas as Classes previstas.

Matriz de Confusão. *os valores foram arredondados.


,0,1
0,2318.0,154.0
1,339.0,445.0
